In [2]:
! explorer .

In [1]:
! code .

In [1]:
import numpy as np

def _angle_between(refvec):
    origin=[0,0]
    ang1 = np.arctan2(*origin[::-1])
    ang2 = np.arctan2(*refvec[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

def create_point_random(x,num):
    """OpenVSP Xsec_point create
    
    Arguments:
        x {float} -- x_start point(Y=0)
        num {int} -- number of xsec_point
    
    Returns:
        list -- Xsec_point_list
    """
    start_and_end_point = [x,0]
    point = np.random.randn(num,2)
    point = np.insert(point,0,start_and_end_point,axis=0)
    pnt=point.tolist()
    pnt = sorted(pnt, key=_angle_between)
    pnt.append(start_and_end_point)
    return pnt

## dummy SUAVE Vehicle( only Fuselage)

In [2]:
import SUAVE
from SUAVE.Core import Units, Data
import vsp
import numpy as np
# from SUAVE import func_tanuki

vehicle = SUAVE.Vehicle()
vehicle.tag = 'test'    

# ------------------------------------------------------------------
#  Fuselage
# ------------------------------------------------------------------

fuselage = SUAVE.Components.Fuselages.Fuselage()
fuselage.tag = 'fuselage'

fuselage.number_coach_seats    = vehicle.passengers
fuselage.seats_abreast         = 6
fuselage.seat_pitch            = 1     * Units.meter
fuselage.fineness.nose         = 1.6
fuselage.fineness.tail         = 2.
fuselage.lengths.nose          = 6.4   * Units.meter
fuselage.lengths.tail          = 8.0   * Units.meter
fuselage.lengths.cabin         = 28.85 * Units.meter
fuselage.lengths.total         = 38.02 * Units.meter
fuselage.lengths.fore_space    = 6.    * Units.meter
fuselage.lengths.aft_space     = 5.    * Units.meter
fuselage.width                 = 3.74  * Units.meter
fuselage.heights.maximum       = 3.74  * Units.meter
fuselage.effective_diameter    = 3.74     * Units.meter
fuselage.areas.side_projected  = 142.1948 * Units['meters**2'] 
fuselage.areas.wetted          = 446.718  * Units['meters**2'] 
fuselage.areas.front_projected = 12.57    * Units['meters**2'] 
fuselage.differential_pressure = 5.0e4 * Units.pascal # Maximum differential pressure

fuselage.heights.at_quarter_length          = 3.74 * Units.meter
fuselage.heights.at_three_quarters_length   = 3.65 * Units.meter
fuselage.heights.at_wing_root_quarter_chord = 3.74 * Units.meter

# add to vehicle
vehicle.append_component(fuselage)


## SUAVE vsp write(only Fuselage)

In [3]:
# -------------
# Fuselage
# -------------    

fuselage = vehicle.fuselages.fuselage
width    = fuselage.width
length   = fuselage.lengths.total
hmax     = fuselage.heights.maximum
height1  = fuselage.heights.at_quarter_length
height2  = fuselage.heights.at_wing_root_quarter_chord 
height3  = fuselage.heights.at_three_quarters_length
effdia   = fuselage.effective_diameter
n_fine   = fuselage.fineness.nose 
t_fine   = fuselage.fineness.tail  
# w_ac     = wing.aerodynamic_center

# w_origin = vehicle.wings.main_wing.origin
# w_c_4    = vehicle.wings.main_wing.chords.root/4.

### fuselageを作成して、全長をセット

In [4]:
fuse_id = vsp.AddGeom("FUSELAGE") 
vsp.SetGeomName(fuse_id, fuselage.tag)
vsp.SetParmVal(fuse_id,"Length","Design",length)
# x1 = n_fine*width/length
# vsp.SetParmVal(fuse_id,"XLocPercent","XSec_1",x1)

38.02

### 仮の断面を作成
辞書型

In [9]:
mydict ={}
Xsec_STA = [0,2,3,5,10,15,18,20,length]
temp = create_point_random(1,3)
for i in Xsec_STA:
    mydict[i] = temp
fuselage.Sections.Xsec=mydict

### 断面の作成
vsp.InsertXSec(fuse_id, 断面の番号,断面のID（6はFUSE_FILE))

fix_secにある断面は移動できないので、これでグルーピングする。グルーピング結果で場合わけ
- インサートする断面とgroupは同じになる
- iはインサートした数（グループに足すことによって、断面の番号を補正している　←断面を増やすとfixsecの番号がずれるため）

In [6]:
# taisho_sec = 3
fix_sec = [0,0.25,0.5,0.75,1]
bins = np.array(fix_sec)
i = 0
for taisho_sec in range(len(Xsec_STA)):
    Xsec = "XSec_" + str(taisho_sec)
    location = Xsec_STA[taisho_sec]/length
    
    if location in fix_sec:
        group = 0
    else:
        group = inds = np.digitize(location, bins)

    if group == 0:
        pass
    else :
        vsp.InsertXSec(fuse_id, int(group+i),6)
        vsp.SetParmVal(fuse_id,"XLocPercent",Xsec,location)
        i += 1

### 断面の形状をセットする

In [10]:
for taisho_sec in range(len(Xsec_STA)):
    Xsec_data = mydict[Xsec_STA[taisho_sec]]
    xsurf_id = vsp.GetXSecSurf(fuse_id, taisho_sec)
    vsp.ChangeXSecShape(xsurf_id, taisho_sec, vsp.XS_FILE_FUSE)

    # get xsec id and set point
    file_xsec_id = vsp.GetXSec(xsurf_id, taisho_sec)
    # Build Point Vec
    pnt_vec = vsp.Vec3dVec()
    for vec in Xsec_data:
        pnt_vec.push_back(vsp.vec3d(vec[0], vec[1], 0.0)) 
    # Load Points Into XSec
    vsp.SetXSecPnts(file_xsec_id, pnt_vec)

### vspデータの出力
エラーだと、vspで読み込みができない

In [11]:
# vsp.WriteVSPFile(fuselage.tag + ".vsp3")
vsp.WriteVSPFile("ssss7" + ".vsp3")

## 工事中

### 一点目と最後はpointとする。